<a href="https://colab.research.google.com/github/dunomi/dunomi/blob/master/projektNAstate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/virus_cases.csv')

USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)


Przygotowanie tabeli z przypadkami Corona virus

In [0]:
USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))

USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]


Przygotowanie tabeli z temperaturą i wilgotnością.

USAdata - tabela z przyrostem przypadków zachorowań, stopniami temperatury oraz procentową wilgotnością.

USAdata2 - tabela z przyrostem przypadków zachorowań, przyrostem temperatury oraz przyrostem wilgotności.

In [0]:
USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100

  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
  
USAdata=pd.merge(USAcases,USAweather)

In [0]:
USAweather2=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][4:])/np.array(USA_temperature_row.loc[i][3:-1])
  wilg=np.array(USA_humidity_row.loc[i][4:])/np.array(USA_humidity_row.loc[i][3:-1])

  USAweather2=USAweather2.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))

USAdata2=pd.merge(USAcases,USAweather2)

USAcumdata - tabela z danymi przygoptowanymi do wykresu dla USAdata

USAcumdata2 - tabela z danymi przygoptowanymi do wykresu dla USAdata2

In [0]:
USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

USAcumdataLess50=USAcumdata[USAcumdata.Przyrosty<50]
#USAcumdataMore50=USAcumdata[USAcumdata.Przyrosty>=50]

In [0]:
USAcumdata2=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata2)):
  stateName=USAdata2.loc[i][0]
  przy=np.cumprod(USAdata2['Przyrosty'][i])[5]
  temp=USAdata2.Temperatura[i].mean()
  wilg=USAdata2.Wilgotnosc[i].mean()
  USAcumdata2=USAcumdata2.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

USA2cumdataLess50=USAcumdata[USAcumdata2.Przyrosty<50]
#USAcumdataMore50=USAcumdata[USAcumdata.Przyrosty>=50]

Wykresy

In [0]:
fig = px.scatter(x=USAcumdataLess50['Temperatura'].tolist(), 
                 y=USAcumdataLess50['Wilgotnosc'].tolist(),
                 size=USAcumdataLess50['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.update_yaxes(range=[0,1])
fig.show()

'''
fig = px.scatter(x=USAcumdataMore50['Temperatura'].tolist(), 
                 y=USAcumdataMore50['Wilgotnosc'].tolist(),
                 size=USAcumdataMore50['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.show()
'''

In [47]:
fig = px.scatter(x=USA2cumdataLess50['Temperatura'].tolist(), 
                 y=USA2cumdataLess50['Wilgotnosc'].tolist(),
                 size=USA2cumdataLess50['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
#fig.update_yaxes(range=[0,1])
fig.show()

In [44]:
USAcumdata2

,Stan,Przyrosty,Temperatura,Wilgotnosc
0,Washington,3.410784,1.212121,1.027368
0,New York,5.555187,0.951454,0.903133
0,California,3.899346,0.955211,1.083732
0,Massachusetts,2.140884,1.059623,1.018179
0,Georgia,7.068903,0.982985,1.036167
0,Colorado,10.670793,1.202778,1.209982
0,Florida,10.357190,1.033338,0.994810
0,New Jersey,11.857651,0.944283,1.125707
0,Oregon,2.588868,1.132096,1.109021
0,Texas,6.570615,1.002792,1.056591
